In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import pandas as pd
from pathlib import Path
import util

In [3]:
dir_data = Path('/Users/josh/Documents/Harvard/GLM/sabatinilab-glm/data/old-data-version/raw-new/Figure_1_2')
dir_output = Path('/Users/josh/Desktop/example_output_folder')

lst_dict_inputdata = [
    {'session_id': 'WT63_11082021',
    'filepath_signal': dir_data / Path('GLM_SIGNALS_WT69_12192021.txt'),
    'filepath_trial': dir_data / Path('GLM_TABLE_WT69_12192021.txt'),
    'bool_trialTable_matlab_indexed': True,
    'columnName_trialTable_trialId': None,
    'columnRenames_signal': {'Ch1': 'gDA', 'Ch5': 'gACH'},
    'columnRenames_trial': {
        'photometryCenterInIndex': 'CI',
        'photometryCenterOutIndex': 'CO',
        'photometrySideInIndex': 'SI',
        'photometrySideOutIndex': 'SO',
        'hasAllPhotometryData': 'hasData'
    }},
]

def construct_lick(df, signal_left_lick_col, signal_right_lick_col):
    return (df[signal_left_lick_col] + df[signal_right_lick_col]).values

def construct_signal_entry(df, signal_left_in_col, signal_right_in_col):
    return (df[signal_left_in_col] + df[signal_right_in_col]).values

# def construct_SEnt(df, signal_entry_col, trial_entry_col):
#     return (df[signal_entry_col] - df[trial_entry_col]).values

dir_output.mkdir(parents=True, exist_ok=True)

columnName_alignment_trial_start = 'CI'
columnName_alignment_trial_end = 'SO'
trial_index_columns = ['CI', 'CO', 'SI', 'SO',]
trial_num_columns = ['nTrial', 'hasData',]
trial_cat_columns = ['wasRewarded', 'word',]
signal_dense_columns = ['gDA', 'gACH',]
signal_sparse_columns = ['lick', 'sideIn']

In [4]:
dict_inputdata = lst_dict_inputdata[0]
df_signal = pd.read_csv(dict_inputdata['filepath_signal'])
df_trial = pd.read_csv(dict_inputdata['filepath_trial'])

In [5]:
# df_trial
# signal_sparse_columns = ['rightLick', 'leftLick', 'SEnt']

In [6]:
df_signal

,Ch1,Ch2,Ch5,Ch6,centerOcc,centerIn,centerOut,rightOcc,rightIn,rightOut,rightLick,leftOcc,leftIn,leftOut,leftLick,reward,noreward
0,-2.515587,0,0.087224,0,0,0,0,0,0,0,0,1,0,0,1,0,1
1,-1.660977,0,-0.159638,0,0,0,0,0,0,0,0,1,0,0,0,0,1
2,-2.155949,0,-0.408390,0,0,0,0,0,0,0,0,1,0,0,1,0,1
3,-0.868106,0,-0.799495,0,0,0,0,0,0,0,0,1,0,0,0,0,1
4,-0.477200,0,-0.422909,0,0,0,0,0,0,0,0,1,0,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29845,-2.426426,0,0.590765,0,0,0,0,0,0,0,0,0,0,0,0,0,0
29846,-2.861054,0,-1.745552,0,0,0,0,0,0,0,0,0,0,0,0,0,0
29847,-2.314100,0,-0.742046,0,0,0,0,0,0,0,0,0,0,0,0,0,0
29848,-1.455169,0,-0.879324,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [7]:
for dict_inputdata in lst_dict_inputdata:
    df_signal = pd.read_csv(dict_inputdata['filepath_signal']).rename(dict_inputdata['columnRenames_signal'], axis=1)
    df_trial = pd.read_csv(dict_inputdata['filepath_trial']).rename(dict_inputdata['columnRenames_trial'], axis=1)
    
    df_trial['nTrial'] = np.arange(df_trial.shape[0])
    df_signal['lick'] = construct_lick(df_signal, 'leftLick', 'rightLick')
    df_signal['sideIn'] = construct_signal_entry(df_signal, 'rightIn', 'leftIn')

    nrows = df_signal.shape[0]
    
    df_trial = df_trial[(df_trial[trial_index_columns] == 0).sum(axis=1) == 0]
    df_trial_indices = df_trial[trial_index_columns]
    
    matrix_signal_sparse = util.Matrix.from_pd_dense_num(df_signal[signal_sparse_columns]).one_hots_to_inx()
    matrix_signal_dense = util.Matrix.from_pd_dense_num(df_signal[signal_dense_columns])
    matrix_trial_inx = util.Matrix.from_pd_sparse_num(nrows, indices=df_trial_indices - 1, values=None, sparse_value=0)
    matrix_trial_num = util.Matrix.from_pd_sparse_num(nrows, indices=df_trial_indices - 1, values=df_trial[trial_num_columns], sparse_value=np.nan)
    matrix_trial_cat = util.Matrix.from_pd_sparse_cat(nrows, indices=df_trial_indices - 1, values=df_trial[trial_cat_columns], sparse_value=np.nan)
    
    matrix_all_preds = util.Matrix([matrix_signal_sparse, matrix_signal_dense, matrix_trial_inx, matrix_trial_num, matrix_trial_cat])

    # # print(matrix_all_preds.vectors['centerOcc'])

    
    # mtc = matrix_trial_cat.to_pd()['nTrial']
    # display(mtc[(~mtc.isna()).sum(axis=1) > 0])
    # # display(matrix_trial_cat.to_pd()['nTrial'])
    # display(matrix_trial_inx.to_pd())

In [8]:
display(matrix_signal_sparse.shift([-1, 0, 1]).to_pd().sort_index(axis=1))

lick           sideIn          
        -1    0    1     -1    0    1
0      0.0  1.0  NaN    0.0  0.0  NaN
1      1.0  0.0  1.0    0.0  0.0  0.0
2      0.0  1.0  0.0    0.0  0.0  0.0
3      1.0  0.0  1.0    0.0  0.0  0.0
4      0.0  1.0  0.0    0.0  0.0  0.0
...    ...  ...  ...    ...  ...  ...
29845  0.0  0.0  0.0    0.0  0.0  0.0
29846  0.0  0.0  0.0    0.0  0.0  0.0
29847  0.0  0.0  0.0    0.0  0.0  0.0
29848  0.0  0.0  0.0    0.0  0.0  0.0
29849  NaN  0.0  0.0    NaN  0.0  0.0

[29850 rows x 6 columns]

In [9]:
matrix_trial_num.to_pd()

,nTrial,hasData,nTrial,hasData,nTrial,hasData,nTrial,hasData
,CI,CI,CO,CO,SI,SI,SO,SO
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
29845,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
29846,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
29847,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
with pd.option_context('display.max_columns', 1000):
    # display(matrix_all_preds.shift(list(range(-20, 21))).to_pd().sort_index(axis=1).ffill())
    print()
    

In [11]:
break

SyntaxError: 'break' outside loop (668683560.py, line 1)

In [ ]:
matrix_all_preds.vectors

In [ ]:
[type(m) for m in matrix_all_preds.vectors.values()]

In [ ]:
mappd = matrix_all_preds.to_pd()

In [ ]:
list(mappd.columns)

In [ ]:
# mappd[('word', 'photometrySideOutIndex')].dropna()
matrix_all_preds.vectors['rightIn']

In [ ]:
from sklearn.linear_model import LinearRegression
lr = LinearRegression()
lr.fit(mappd[[]], mappd['Ch5'])

In [ ]:
display(matrix_signal_dense.to_pd())
display(matrix_trial_inx.to_pd())
display(matrix_trial_num.to_pd())
display(matrix_trial_cat.to_pd())